In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import csv
from googleapiclient.discovery import build
import datetime
import math
import time
from urllib.parse import urlparse
import os
from IPython.display import HTML
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

In [ ]:
eu = pd.read_csv("eu_checked20250826.csv")

In [ ]:
eu

In [ ]:
# read in API key and case ID
api_key = open('api_key.txt', 'r').read().strip()
case_id = open('case_id.txt', 'r').read().strip()

In [ ]:
def google(search_term, api_key, case_id):
    """
    Use Google Custom Search API to collect search results.
    
    Args:
        search_term: search string. The maximium length is 2048 characters.
        api_key: api key.
        case_id: case_id.
    Returns:
        titlel: the title of each returned search result.
        linkl: the link of each returned search result.
        snippetl: the snippet of each returned search result.
    """
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=search_term, cx=case_id).execute()
    est_total_num = int(result["searchInformation"]["totalResults"])
    titlel = []
    linkl = []
    snippetl = []
    if est_total_num == 0:
        return titlel, linkl, snippetl
    elif est_total_num <= 10:
        for item in result.get("items", []):
            titlel.append(item["title"])
            linkl.append(item["link"])
            snippetl.append(item['snippet'])
        return titlel, linkl, snippetl
    else:
        for item in result.get("items", []):
            titlel.append(item["title"])
            linkl.append(item["link"])
            snippetl.append(item['snippet'])
        total_page = math.ceil(est_total_num/10)
        if total_page > 10:
            total_page = 10
        for page in range(1, total_page):
            start = page * 10 + 1
            more_result = service.cse().list(q=search_term, cx=case_id, start=start).execute()
            new_total_num = int(more_result["searchInformation"]["totalResults"])
            if new_total_num == 0:
                return titlel, linkl, snippetl
            else:
                for item in more_result.get("items", []):
                    titlel.append(item["title"])
                    linkl.append(item["link"])
                    snippetl.append(item['snippet'])
        return titlel, linkl, snippetl

In [ ]:
# define the output file name
today_str = datetime.datetime.now().strftime("%Y%m%d")
filename = f"data/cityweb{today_str}.csv"

In [ ]:
# Controls for subset selection
TEST_LIMIT = None         
START_AT = 470              
SLEEP_BETWEEN_CITIES = 1.0 
SLEEP_ON_ERROR = 60        
VERBOSE = True

# define date and ID counter
idx = 5816
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
subset = eu.iloc[START_AT: START_AT + TEST_LIMIT] if TEST_LIMIT else eu.iloc[START_AT:]
total_rows = len(subset)

print(f"Starting city web search on {total_rows} row(s). Output -> {filename}", flush=True)

results_written = 0

with open(filename, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)

    for row_idx, (_, row) in enumerate(subset.iterrows(), start=1):
        domain = ""
        try:
            keyword = str(row.get("ai_translate", "")).strip()
            source_url = str(row.get("City_Website", "")).strip()

            # record city/state/country exactly as in the original file
            city_val = str(row.get("City_Name", ""))
            state_val = str(row.get("State", ""))
            country_val = str(row.get("Country_Name", ""))

            # basic validation
            if not keyword or not source_url:
                if VERBOSE:
                    print(f"[{row_idx}/{total_rows}] Skipping (missing keyword or City_Website) — {city_val}, {state_val}, {country_val}", flush=True)
                time.sleep(SLEEP_BETWEEN_CITIES)
                continue

            parsed = urlparse(source_url)
            domain = parsed.netloc + parsed.path if parsed.netloc else re.sub(r'^https?://', '', source_url).strip('/')
            if not domain:
                if VERBOSE:
                    print(f"[{row_idx}/{total_rows}] Skipping (invalid domain) — {city_val}, {state_val}, {country_val} — raw: {source_url}", flush=True)
                time.sleep(SLEEP_BETWEEN_CITIES)
                continue

            search_term = f'"{keyword}" site:{domain}'

            if VERBOSE:
                print(f"[{row_idx}/{total_rows}] {city_val}, {state_val}, {country_val} | domain={domain} | query='{search_term}'", flush=True)

            titlel, linkl, snippetl = google(search_term, api_key, case_id)
            if VERBOSE:
                print(f"    -> {len(titlel)} result(s)", flush=True)

            if len(titlel) == 0:
                time.sleep(SLEEP_BETWEEN_CITIES)
                continue

            for title, link, snippet in zip(titlel, linkl, snippetl):
                id_str = f'cityweb{idx:06d}'
                hyperlink = f'=HYPERLINK("{link}", "{link}")'
                writer.writerow([id_str, date_str, city_val, state_val, country_val, title, snippet, hyperlink])
                idx += 1
                results_written += 1

            time.sleep(SLEEP_BETWEEN_CITIES)

        except Exception as e:
            print(f"[{row_idx}/{total_rows}] Error for {city_val}, {state_val}, {country_val} (domain={domain or 'unknown'}): {e} — sleeping {SLEEP_ON_ERROR}s then retrying once", flush=True)
            time.sleep(SLEEP_ON_ERROR)
            # retry once
            try:
                titlel, linkl, snippetl = google(search_term, api_key, case_id)
                if VERBOSE:
                    print(f"    (retry) -> {len(titlel)} result(s)", flush=True)
                for title, link, snippet in zip(titlel, linkl, snippetl):
                    id_str = f'cityweb{idx:06d}'
                    hyperlink = f'=HYPERLINK("{link}", "{link}")'
                    writer.writerow([id_str, date_str, city_val, state_val, country_val, title, snippet, hyperlink])
                    idx += 1
                    results_written += 1
                print("    (retry) Done", flush=True)
                time.sleep(SLEEP_BETWEEN_CITIES)
            except Exception as e2:
                print(f"    (retry) failed: {e2}", flush=True)

print(f"Finished. Wrote {results_written} row(s) to {filename}.", flush=True)


In [ ]:
df = pd.read_csv(filename, names=["id", "search_date","city","state","country","link_title","link_des","link"])

In [ ]:
df["link"] = df["link"].str.extract(r'HYPERLINK\("([^"]+)"')
df

In [ ]:
df

In [ ]:
# prepare for coding
new_cols = [
    "inaccessible",
    "irrelevant",
    "year",
    "use_case",
    "mode",
    "motivation",
    "stakeholder",
    "detail",
    "note",
    "other_ref",
    "coder"
]

for col in new_cols:
    df[col] = ""

In [ ]:
# reorder
df = df[["id","search_date","link_title","link_des","link",
         "inaccessible","irrelevant",
         "city","state","country",
         "year","use_case","mode","motivation","stakeholder","detail","note","other_ref","coder" ]]

In [ ]:
df

In [ ]:
filename_check = filename.replace(".csv", "_check.xlsx")
df.to_excel(filename_check, index=False)

In [ ]:
# set up the mode coding options
wb = load_workbook(filename_check)
ws = wb["Sheet1"]

In [ ]:
dv = DataValidation(
    type="list",
    formula1='"road,rail,air,waterborne,cross_modal"',
    allow_blank=True
)

In [ ]:
dv.add("M2:M10000")
ws.add_data_validation(dv)

In [ ]:
wb.save(filename_check)